In [1]:
import sys 
sys.path.append('../')
import pandas as pd 
import numpy as np 
import talib
import sqlite3 as sql
import matplotlib.pyplot as plt 
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, a in enumerate(my_assets):
    print(i, a)

0 EUR/USD
1 USD/JPY
2 GBP/USD
3 USOil
4 SPX500
5 VOLX
6 XAU/USD
7 Bund
8 BTC/USD


In [4]:
conn = sql.connect('../PriceData/PriceData_m5.db')
ticker = my_assets[0]
print(ticker)
data_sql = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
for col in ['open', 'close', 'high', 'low']:
    data_sql[col] = data_sql[['bid'+col, 'ask'+col]].mean(axis=1)
conn.close()
data_sql.tail()

EUR/USD


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,open,close,high,low
date,,,,,,,,,,,,,
2022-07-08 20:35:00,1.01827,1.01813,1.01828,1.01813,1.01842,1.01826,1.01845,1.01825,115,1.018345,1.018195,1.018365,1.018190
2022-07-08 20:40:00,1.01813,1.01822,1.01829,1.01809,1.01825,1.01835,1.01842,1.01824,104,1.018190,1.018285,1.018355,1.018165
2022-07-08 20:45:00,1.01821,1.01824,1.01831,1.01813,1.01835,1.01839,1.01847,1.01828,140,1.018280,1.018315,1.018390,1.018205
2022-07-08 20:50:00,1.01825,1.01824,1.01832,1.01820,1.01839,1.01844,1.01850,1.01837,217,1.018320,1.018340,1.018410,1.018285
2022-07-08 20:55:00,1.01824,1.01853,1.01854,1.01824,1.01845,1.01882,1.01885,1.01845,202,1.018345,1.018675,1.018695,1.018345


In [6]:
params = {'EUR/USD': {'slope':100, 'mama':0.8, 'tsf':100},
          'USD/JPY': {'slope':100, 'mama':0.2, 'tsf':250},
          'GBP/USD': {'slope':350, 'mama':0.5, 'tsf':100},
          'USOil': {'slope':100, 'mama':0.1, 'tsf':250},
          'SPX500': {'slope':400, 'mama':0.5, 'tsf':250},
          'VOLX': {'slope':150, 'mama':0.2, 'tsf':400},
          'XAU/USD': {'slope':100, 'mama':0.2, 'tsf':250},
          'Bund': {'slope':400, 'mama':0.1, 'tsf':450},
          'BTC/USD': {'slope':50, 'mama':0.3, 'tsf':300}}

In [25]:
for ticker in my_assets:

    conn = sql.connect('../PriceData/PriceData_m5.db')
    print(ticker)
    data = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
    for col in ['open', 'close', 'high', 'low']:
        data[col] = data[['bid'+col, 'ask'+col]].mean(axis=1)
    conn.close()
    

    data['ht'] = talib.HT_TRENDLINE(data.close)
    data['mama'], data['fama'] = talib.MAMA(data.ht, fastlimit=params[ticker]['mama'], 
                                            slowlimit=params[ticker]['mama']/10)
    data['slope'] = talib.LINEARREG_SLOPE(data.close, timeperiod=params[ticker]['slope'])
    data['tsf'] = talib.TSF(data.ht, params[ticker]['tsf'])
    data.dropna(axis=0, inplace=True)

    # signals
    data['mama_signals'] = np.where(data.mama > data.fama, 1, -1) 
    data['slope_signals'] = data.slope.apply(np.sign)
    data['tsf_signals'] = np.where(data.ht > data.tsf, 1, -1)
    signal_cols = ['mama_signals', 'slope_signals', 'tsf_signals']
    data['agg_signals'] = data[signal_cols].mode(axis=1)
    
    print(f"{data.index[0]} - {data.index[-1]} | {data.index[-1]-data.index[0]}")
    backtest = IterativeBacktester(data=data,
                                   signals=data.agg_signals,
                                   freq='m5')

    backtest.backtest(progress_bar=False)
    # backtest.results()
    print(f"{backtest.return_df.loc['TotalReturn', 'Portfolio']*100:.4f}%")

    print("Signals")
    print(backtest.signals.value_counts())

    print("Signals change")
    print(backtest.signals.diff(1).dropna().apply(np.abs).value_counts())

    print("-"*50)

EUR/USD
2022-05-23 20:30:00 - 2022-07-08 20:55:00 | 46 days 00:25:00
6.6168%
Signals
-1.0    5090
 1.0    4748
Name: agg_signals, dtype: int64
Signals change
0.0    9591
2.0     246
Name: agg_signals, dtype: int64
--------------------------------------------------
USD/JPY
2022-05-24 06:35:00 - 2022-07-08 20:55:00 | 45 days 14:20:00
4.5242%
Signals
 1.0    5666
-1.0    4022
Name: agg_signals, dtype: int64
Signals change
0.0    9585
2.0     102
Name: agg_signals, dtype: int64
--------------------------------------------------
GBP/USD
2022-05-24 09:10:00 - 2022-07-08 20:55:00 | 45 days 11:45:00
5.5638%
Signals
-1.0    5240
 1.0    4411
Name: agg_signals, dtype: int64
Signals change
0.0    9452
2.0     198
Name: agg_signals, dtype: int64
--------------------------------------------------
USOil
2022-05-20 07:05:00 - 2022-07-08 20:40:00 | 49 days 13:35:00
6.9803%
Signals
 1.0    5292
-1.0    4396
Name: agg_signals, dtype: int64
Signals change
0.0    9586
2.0     101
Name: agg_signals, dtype: